# Αναγνώριση Προτύπων Μηχανικής Μάθησης
Εργασία 2 \\
Χρήστος Μπριστογιάννης sdi1900129 \\

##Ερώτημα 1: Αναγνώριση Προσώπων (Face recognition)

###Ζητούμενα i
Να γράψετε μία συνάρτηση loadImages(path, set_number) η οποία παίρνει ως είσοδο το
path στο οποίο βρίσκεται ο φάκελος των εικόνων π.χ. loadImages(“C:/images”, “Set_1”),
διαβάζει τις εικόνες και επιστέφει έναν πίνακα δεδομένων ανάλογα με το set_number,
όπου κάθε εικόνα αναπαρίσταται ως διάνυσμα στήλη. Η συνάρτηση επιστέφει επίσης τις
κατηγορίες (labels) στις οποίες ανήκουν οι διαφορετικές εικόνες κωδικοποιημένες με
ακεραίους (π.χ. 0 για φωτογραφίες που ανήκουν στο person_0, 1 για τις φωτογραφίες
που ανήκουν στο person_1 κτλ).

In [ ]:
import cv2
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

def loadImages(path, set_number):
  if set_number == "Set_1":
    start = 1
    end = 7
  elif set_number == "Set_2":
    start = 8
    end = 19
  elif set_number == "Set_3":
    start = 20
    end = 31
  elif set_number == "Set_4":
    start = 32
    end = 45
  elif set_number == "Set_5":
    start = 46
    end = 64
  else:
    print("Error")
    return

  arr = []
  labels = []
  for i in range(1, 11, 1):
    for j in range(start, end+1, 1):
      if i == 10:
        mes = 'person10'+'_'
      else:
        mes = 'person0'+str(i)+'_'
      if j < 10:
        mes = mes+'0'+str(j)
      else:
        mes = mes + str(j)

      img = cv2.imread(path+ mes + '.png', 0)

      arr.append(np.reshape(img,-1))
      labels.append(i)
  return arr, labels


#load images
numberOfSets = 5
nmsset = ["Set_1", "Set_2", "Set_3", "Set_4", "Set_5"]
imgset = [None] * numberOfSets
lblset = [None] * numberOfSets
path = '/content/drive/My Drive/faces/'

for i in range(0, numberOfSets):
  imgset[i], lblset[i] = loadImages(path, nmsset[i])
  # print(imgset[i])

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


###Ζητούμενα ii
Να εκπαιδεύσετε την μέθοδο Eigenfaces με d = 9 και d = 30 χρησιμοποιώντας όλες τις
εικόνες στο Set_1 (70 εικόνες) και να αναγνωρίσετε τα πρόσωπα στα Set_1 έως Set_5.
Για κάθε Set και κάθε τιμή της διάστασης d να αναφέρετε την ακρίβεια ταξινόμησης. Για
το Set_1 αναμένουμε 100% ακρίβεια ταξινόμησης καθώς χρησιμοποιήθηκε για την
εκπαίδευση της μεθόδου Eigenfaces. Σχολιάστε την δυνατότητα γενίκευσης της μεθόδου
στα διαφορετικά Sets.

In [ ]:
from sklearn.decomposition import PCA


n_components = 9 ## how may componets to keep
pca = PCA(n_components=n_components, whiten=True).fit(imgset[4])

print ("\n--pca--\n",pca)


components = pca.transform(imgset[4]) ## tranform the data to the new axes system)
print ("components.shape",components.shape)


--pca--
 PCA(n_components=9, whiten=True)
components.shape (190, 9)


##Ερώτημα 2: Ταξινόμηση εικόνων χρησιμοποιώντας SVMs

In [2]:
from sklearn.datasets import fetch_openml

# Φόρτωσει δεδομένων
mnist = fetch_openml('mnist_784')

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:100: UserWarning: A network error occurred while downloading https://openml.org/data/v1/download/52667. Retrying...
  warn(


In [3]:
import numpy as np
import random

# Μετατρέπουμε τις εικόνες σε μορφή διανύσματος διαστάσεων 28 x 28
X = np.array(mnist.data)
Y = np.array(mnist.target)
X = X.reshape(-1, 28, 28)

# Κανονικοποιούμε τα δεδομένα στο διάστημα [0,1]
X = X / 255.0


def split_dataset(data, labels, train_size, val_size, test_size):
  combined = list(zip(data, labels))  # Combine the lists into pairs
  random.shuffle(combined)  # Shuffle the combined list
  shuffled_data, shuffled_labels = zip(*combined)

  # Split the shuffled data and labels
  trainData = np.array(shuffled_data[:train_size])
  trainLbl = np.array(shuffled_labels[:train_size])
  valData = np.array(shuffled_data[train_size:train_size + val_size])
  valLbl = np.array(shuffled_labels[train_size:train_size + val_size])
  testData = np.array(shuffled_data[train_size + val_size:])
  testLbl = np.array(shuffled_labels[train_size + val_size:])

  return trainData, trainLbl, valData, valLbl, testData, testLbl

trainData, trainLbl, valData, valLbl, testData, testLbl = split_dataset(X, Y, 1000, 500, 500)


In [6]:
from sklearn import svm
from sklearn.metrics import accuracy_score
import time

# Define the hyperparameters to test
c_values = [0.01, 0.1, 1, 10, 100, 1000]
gamma_values = [0.0001, 0.001, 0.01, 0.1, 1, 10]
sum_execution = 0

# Train and evaluate SVMs with different hyperparameter combinations
for kernel in ['linear', 'rbf']:
    best_accuracy = 0
    best_c = None
    best_gamma = None
    
    print(f'Testing {kernel} kernel...')

    for c in c_values:
        for gamma in gamma_values:
            # Train SVM
            svm_model = svm.SVC(kernel=kernel, C=c, gamma=gamma)

            # Measure execution time
            start_time = time.time()
            svm_model.fit(trainData.reshape(trainData.shape[0], -1), trainLbl)
            execution_time = time.time() - start_time
            sum_execution += execution_time
            
            # Evaluate SVM on validation set
            val_predictions = svm_model.predict(valData.reshape(valData.shape[0], -1))
            val_accuracy = accuracy_score(valLbl, val_predictions)
            
            print(f' for values C={c} and gamma={gamma} the accuracy is {val_accuracy} and execution time is {execution_time}')
            
            # Update best hyperparameters if accuracy is higher
            if val_accuracy > best_accuracy:
                best_accuracy = val_accuracy
                best_c = c
                best_gamma = gamma

    # Train SVM with best hyperparameters on full training set
    svm_model = svm.SVC(kernel=kernel, C=best_c, gamma=best_gamma)
    svm_model.fit(trainData.reshape(trainData.shape[0], -1), trainLbl)

    # Evaluate SVM on test set
    test_predictions = svm_model.predict(testData.reshape(testData.shape[0], -1))
    test_accuracy = accuracy_score(testLbl, test_predictions)

    # Print results
    print(f'Best hyperparameters for {kernel} kernel: C={best_c}, gamma={best_gamma} and average execution time {sum_execution/(len(c_values)*len(gamma_values))}')
    print(f'Validation accuracy: {best_accuracy:.4f}')
    print(f'Test accuracy: {test_accuracy:.4f}\n')
    sum_execution = 0


Testing linear kernel...
 for values C=0.01 and gamma=0.0001 the accuracy is 0.886 and execution time is 0.1956005096435547
 for values C=0.01 and gamma=0.001 the accuracy is 0.886 and execution time is 0.18890905380249023
 for values C=0.01 and gamma=0.01 the accuracy is 0.886 and execution time is 0.19306015968322754
 for values C=0.01 and gamma=0.1 the accuracy is 0.886 and execution time is 0.18667197227478027
 for values C=0.01 and gamma=1 the accuracy is 0.886 and execution time is 0.18484807014465332
 for values C=0.01 and gamma=10 the accuracy is 0.886 and execution time is 0.18763494491577148
 for values C=0.1 and gamma=0.0001 the accuracy is 0.902 and execution time is 0.14275479316711426
 for values C=0.1 and gamma=0.001 the accuracy is 0.902 and execution time is 0.14754605293273926
 for values C=0.1 and gamma=0.01 the accuracy is 0.902 and execution time is 0.14212965965270996
 for values C=0.1 and gamma=0.1 the accuracy is 0.902 and execution time is 0.13668227195739746
 

In [7]:
from sklearn.decomposition import PCA

# Define the variance ratios to test
variance_ratios = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 0.95]

# Perform PCA and SVM for different variance ratios
for variance_ratio in variance_ratios:
    print(f'Testing PCA with variance ratio {variance_ratio}...')

    # Perform PCA
    pca = PCA(n_components=variance_ratio)
    pca_trainData = pca.fit_transform(trainData.reshape(trainData.shape[0], -1))
    pca_valData = pca.transform(valData.reshape(valData.shape[0], -1))
    pca_testData = pca.transform(testData.reshape(testData.shape[0], -1))
    components = pca.n_components_

    # Train SVM with best hyperparameters on PCA-transformed data
    svm_model = svm.SVC(kernel=kernel, C=best_c, gamma=best_gamma)
    
    # Measure execution time
    start_time = time.time()
    svm_model.fit(pca_trainData, trainLbl)
    execution_time = time.time() - start_time

    # Evaluate SVM on validation set
    val_predictions = svm_model.predict(pca_valData)
    val_accuracy = accuracy_score(valLbl, val_predictions)

    # Evaluate SVM on test set
    test_predictions = svm_model.predict(pca_testData)
    test_accuracy = accuracy_score(testLbl, test_predictions)

    # Print results
    print(f'Number of components: {components}')
    print(f'Validation accuracy: {val_accuracy:.4f}')
    print(f'Test accuracy: {test_accuracy:.4f}')
    print(f'Execution time: {execution_time:.4f} seconds\n')


Testing PCA with variance ratio 0.1...
Number of components: 2
Validation accuracy: 0.4340
Test accuracy: 0.4195
Execution time: 0.0563 seconds

Testing PCA with variance ratio 0.2...
Number of components: 3
Validation accuracy: 0.4500
Test accuracy: 0.4511
Execution time: 0.0555 seconds

Testing PCA with variance ratio 0.3...
Number of components: 5
Validation accuracy: 0.6760
Test accuracy: 0.7016
Execution time: 0.0827 seconds

Testing PCA with variance ratio 0.4...
Number of components: 7
Validation accuracy: 0.8140
Test accuracy: 0.8119
Execution time: 0.0382 seconds

Testing PCA with variance ratio 0.5...
Number of components: 11
Validation accuracy: 0.8960
Test accuracy: 0.8756
Execution time: 0.0960 seconds

Testing PCA with variance ratio 0.6...
Number of components: 16
Validation accuracy: 0.9040
Test accuracy: 0.8975
Execution time: 0.0406 seconds

Testing PCA with variance ratio 0.7...
Number of components: 25
Validation accuracy: 0.9220
Test accuracy: 0.9136
Execution time